In [0]:
dbutils.widgets.removeAll()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [0]:
dbutils.widgets.text("catalogo", "catalog_jptq")
dbutils.widgets.text("esquema_source", "bronze_jptq")
dbutils.widgets.text("esquema_stage", "silver_jptq")

In [0]:
catalogo = dbutils.widgets.get("catalogo")
esquema_source = dbutils.widgets.get("esquema_source")
esquema_stage = dbutils.widgets.get("esquema_stage")

In [0]:
def fn_transform_genero_pelicula(catalogo: str, esquema_source: str):
   sql_select_genero = f"""SELECT
                        DISTINCT
                        UPPER(SUBSTR(DES_GENERO_PELICULA, 1, 3)) AS COD_GENERO_PELICULA,
                        UPPER(SPLIT_PART(DES_GENERO_PELICULA, ',', 1)) AS DES_GENERO_PELICULA
                        FROM {catalogo}.{esquema_source}.TBL_PELICULAS
                        ORDER BY DES_GENERO_PELICULA ASC"""
   df_t_genero_pelicula = spark.sql(sql_select_genero)
   return df_t_genero_pelicula

In [0]:
df_tmp_genero_pelicula = fn_transform_genero_pelicula(catalogo, esquema_source)

In [0]:
df_tmp_genero_pelicula = df_tmp_genero_pelicula.withColumn("FEC_CARGA", current_timestamp())

In [0]:
display(df_tmp_genero_pelicula)

In [0]:
df_tmp_genero_pelicula.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema_stage}.TBL_TMP_GENERO_PELICULA")

In [0]:
%sql
SELECT
*
FROM
catalog_jptq.silver_jptq.tbl_tmp_genero_pelicula